In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My Drive/Colab Notebooks/tony_chain

/content/drive/My Drive/Colab Notebooks/tony_chain


In [3]:
%ls

app_by_junho           config.py            prepare_sembot.py  utils.py
app_by_tony.ipynb      local_model/         __pycache__/       vectorstore_manager.py
app_by_tony_multiUser  local_model_NCSOFT/  sembot.py          vector_store_path_jhgan/
chain_logging.py       model_manager.py     sembot_test.ipynb


In [4]:
!pip install transformers langchain langchain-community langchain-huggingface fastapi uvicorn pypdf accelerate python-dotenv faiss-gpu pdfplumber pyngrok

In [5]:
!ngrok authtoken 2oQD6abMRFtGqvuk0x3rONRKN9M_YvTRpFjyQ7NTeeeNJXyB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from sembot import Sembot
from prepare_sembot import SembotDocs

pre_sembot = SembotDocs()
sembot = Sembot()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading existing vectorstore...
Loading existing Local Model...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
model = sembot.get_model()
streamer = sembot.get_streamer()

In [10]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import asyncio
from pydantic import BaseModel
from typing import List, Dict, Any
import nest_asyncio
from pyngrok import ngrok
from threading import Thread
from asyncio import Semaphore
from fastapi.responses import JSONResponse

app = FastAPI()

# CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["X-Request-ID"],
)

class SearchRequest(BaseModel):
    question: str

class QueryRequest(BaseModel):
    memory: List[Dict[str, Any]]
    question: str

async def async_stream_response(input_data: dict):
    try:
        # 세마포어를 사용하여 LLM 접근 제어
        async with llm_semaphore:
            prompt = pre_sembot.make_prompt(input_data)
            generation_kwargs = sembot.inference(prompt)
            thread = Thread(target=model.generate, kwargs=generation_kwargs)
            thread.start()

            for new_text in streamer:
                yield new_text.replace("\n", "<br>").replace("<|eot_id|>", "")
                await asyncio.sleep(0.01)

            thread.join()
    except Exception as e:
        # 에러 발생 시 세마포어가 확실히 해제되도록 처리
        print(f"Error in stream_response: {str(e)}")
        raise

@app.post("/search")
async def search_documents(query: SearchRequest):
    try:
        docs = pre_sembot.docs_invoke(query.question)
        return {"docs": docs}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate")
async def stream_answer(query: QueryRequest):
    try:
        # 현재 LLM이 사용 중인지 확인
        if llm_semaphore.locked():
            return JSONResponse(
                status_code=503,
                content={
                    "message": "서버가 현재 다른 요청을 처리 중입니다. 잠시 후 다시 시도해주세요."
                }
            )

        input_data = {
            "memory": query.memory,
            "question": query.question
        }
        return StreamingResponse(
            async_stream_response(input_data),
            media_type="text/plain"
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# 서버 시작 전 초기화
@app.on_event("startup")
async def startup_event():
    global llm_semaphore
    llm_semaphore = Semaphore(1)

# 서버 종료 시 정리
@app.on_event("shutdown")
async def shutdown_event():
    # 필요한 정리 작업 수행
    pass

# 서버 실행
if __name__ == "__main__":
    ngrok_tunnel = ngrok.connect(8001)
    print('Public URL:', ngrok_tunnel.public_url)
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8001)

<ipython-input-10-5765f9f09198>:84: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
<ipython-input-10-5765f9f09198>:90: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("shutdown")


Public URL: https://0d17-34-30-37-93.ngrok-free.app


INFO:     Started server process [16121]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     211.192.252.28:0 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "OPTIONS /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /generate HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


INFO:     211.192.252.28:0 - "POST /search HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     211.192.252.28:0 - "POST /generate HTTP/1.1" 503 Service Unavailable


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16121]
